In [1]:
import os
os.environ["OPENAI_API_KEY"] = "sk-xxx"

In [22]:
k = 5

# Vector similarity recommendation

In [23]:
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
    "bolt://localhost:7687",
  auth=basic_auth("neo4j", "$$1234bala"))

In [24]:
current_title = "The Matrix"
# current_title = "The Great Gatsby"

vector_similarity = '''
match (m:Movie { title: $Title })
call db.index.vector.queryNodes('desc-embeddings', $top_k, m.embedding)
yield node AS similarDesc, score
where  similarDesc <> m
return similarDesc.title as title, similarDesc.description as desc, score
'''

In [25]:
with driver.session(database="neo4j") as session:
  results = session.execute_read(lambda tx: tx.run(vector_similarity, Title=current_title, top_k=k).data() )
  #results = session.read_transaction(lambda tx: tx.run(vector_similarity, top_k=k).data() )
  context = "Related Movies: \n\n" + "\n\n".join(["title: " + record['title'] + "\n" + "Description: " + record['desc'] + "\n" for record in results ])

driver.close()

print(context)

Related Movies: 




In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0)

template = """I've read an article titled {current_movie_title}.
Tell me what should I read next and a brief description of why based only on the provided context.

{context}
"""
prompt = PromptTemplate.from_template(template)

theprompt = prompt.format_prompt(current_movie_title=current_title,context=context)

#print(theprompt)

llm(theprompt.to_messages())

# Graph similarity recommendation

In [ ]:
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
    "bolt://localhost:7687",
  auth=basic_auth("neo4j", "$$1234bala"))

In [ ]:
current_title = "Performance Testing Neo4j Database using Bolt Protocol in Apache JMeter"
current_uri = "https://dev.to/qainsights/performance-testing-neo4j-database-using-bolt-protocol-in-apache-jmeter-1oa9"

graph_similarity = '''
match (a:Article { uri: $uri})-[rt1:refers_to]->(c1)
call n10s.sim.pathsim.search(c1,0.2,{ simulateRoot: false }) yield node as relatedTopic
with a,c1,collect(relatedTopic) + [c1] as topics
unwind topics as c2
match (similarArticle:Article )-[rt2:refers_to]->(c2) //where similarArticle <> a
with a.title as original, similarArticle.title as similar,
[x in collect(n10s.sim.pathsim.value(c1,c2, { simulateRoot: false})) where x > 0 ] as sims ,
collect(n10s.sim.pathsim.path(c1,c2, { simulateRoot: false})) as paths,
collect(["the original article mentions explicitly " + nodes(n10s.sim.pathsim.path(c1,c2, { simulateRoot: false}))[0].prefLabel ]  +
 [" the recommended article mentions explicitly " + nodes(n10s.sim.pathsim.path(c1,c2, { simulateRoot: false}))[-1].prefLabel]  + [r in relationships(n10s.sim.pathsim.path(c1,c2, { simulateRoot: false})) | startNode(r).prefLabel + " is a type of " + endNode(r).prefLabel]) as paths_as_text
where sims <> []
return similar as title, apoc.coll.avg(sims) as sim, sims, reduce(result="", x in paths_as_text | result + reduce(inner="", y in x | inner + "\n"+y))  as explanation
order by sim desc limit 4
'''
     

In [ ]:
with driver.session(database="neo4j") as session:
  results = session.read_transaction(
    lambda tx: tx.run(graph_similarity,
                      uri=current_uri).data())

  context = "Related documents: \n\n" + "\n\n".join(["title: " + record['title'] + "\n" + "explanation: " + record['explanation'] + "\n" for record in results ])

driver.close()

print(context)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0)

template = """I've read an article titled {current_article_title}.
Tell me what should I read next and a brief description of why based only on the provided context.

{context}
"""
prompt = PromptTemplate.from_template(template)

theprompt = prompt.format_prompt(current_article_title=current_title,context=context)

#print(theprompt)

llm(theprompt.to_messages())